In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
import csv
from tqdm.auto import tqdm
import re
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler , PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor ,GradientBoostingRegressor , \
        AdaBoostRegressor , HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
import statsmodels.api as sm
from xgboost import XGBRegressor
from Feature_engineering.FE import *
from Feature_engineering.utils import *
from Feature_engineering.ProjectPaths import *
import warnings
import time
from joblib import dump, load
warnings.filterwarnings('ignore', category=FutureWarning)

In [4]:
# golden_df=pd.read_csv(join(output_folder_paths,"golden.csv"))
# eugene_df=pd.read_csv(join(output_folder_paths,"eugene.csv"))
# cocoa_df=pd.read_csv(join(output_folder_paths,"cocoa.csv"))

In [5]:
# main_df=pd.concat([golden_df,eugene_df,cocoa_df],axis=0)

In [6]:
# main_df["filename"]=main_df["filename"].apply(lambda x : x.split("_")[-1])

In [7]:
# main_df,columns,lickage_features=feature_eng(main_df)

In [8]:
# len(main_df)

In [11]:
# df=main_df[["GHI","DNI","POA_CMP22","Mod_Temp","Cab_Temp","Dry_Temp","Pressure","RH","QA_residual","Imp (A)","Pmp (W)","Vmp (V)","filename"]]
# df.to_csv("./Dataset/main_df.csv",index=False)
# mSi0247_df=main_df[main_df["filename"]=="mSi0247"].drop(columns=["filename"])
# CIGS1_001_df=main_df[main_df["filename"]=="CIGS1-001"].drop(columns=["filename"])
# mSi460BB_df=main_df[main_df["filename"]=="mSi460BB"].drop(columns=["filename"])
# mSi0247_df.to_csv("./Dataset/mSi0247.csv",index=False)
# CIGS1_001_df.to_csv("./Dataset/CIGS1-001.csv",index=False)
# mSi460BB_df.to_csv("./Dataset/mSi460BB.csv",index=False)

main_df=pd.read_csv("./Dataset/main_df.csv")
len(main_df)

769595

In [12]:
sites=main_df["filename"].unique()

In [13]:
X=main_df[["GHI","DNI","POA_CMP22","Mod_Temp","Cab_Temp","Dry_Temp","Pressure","RH","QA_residual","filename"]]
Y=main_df[["Imp (A)","Pmp (W)","Vmp (V)","filename"]]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=X["filename"])

In [14]:
x_train_sites=split_by_site(x_train,sites)
x_test_sites=split_by_site(x_test,sites)
y_train_sites=split_by_site(y_train,sites)
y_test_sites=split_by_site(y_test,sites)

In [15]:
xg_boost_params = {
    "n_estimators": 500,
    "max_depth": 3,
    "min_samples_split": 7,
    "learning_rate": 0.01,
    "loss": "squared_error",
    "verbose":0
}
l1_xg_boost_params={
    "objective":'reg:squarederror',
    "n_estimators": 500,
    "max_depth": 3,
    "learning_rate": 0.01,
    "reg_alpha":2.0,
}
forest_params={
    "max_depth":10, 
    "random_state":0,
    "min_samples_split":10,
    "min_samples_leaf":10, 
    "max_features":'sqrt'
}

# training_labels=["Imp (A)","Pmp (W)","Vmp (V)"]

In [16]:
ip_features=["GHI","DNI","POA_CMP22","Mod_Temp"]
ip_labels=["Imp (A)","Pmp (W)"]
v_features=["Cab_Temp","Dry_Temp","Mod_Temp","Pressure","RH","QA_residual","GHI","DNI"]
preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),ip_features)
])
v_preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),v_features+["i","p"]) #list(range(len(v_features)+2))
])
def multi_train(reg_model,v_reg_model,name=None,stat_rep=False,save=False):

    vmp_test_r2_ls=[]
    vmp_train_r2_ls=[]
    adj_vmp_test_r2_ls=[]
    adj_vmp_train_r2_ls=[]
    pred_time_ls=[]
    for solar_model in x_train_sites:

        x_tr=x_train_sites[solar_model]
        x_te=x_test_sites[solar_model]
        y_tr=y_train_sites[solar_model]
        y_te=y_test_sites[solar_model]

        reg_model.fit(x_tr[ip_features], y_tr[ip_labels])
        ip_preds=reg_model.predict(x_tr[ip_features])
        ip_preds=pd.DataFrame(np.concatenate((ip_preds,x_tr[v_features].to_numpy()),axis=1),
                              columns=v_features+["i","p"])
        v_reg_model.fit(ip_preds,y_tr["Vmp (V)"])
        
        print(f"for site {solar_model}")
        
        test_ip_preds=reg_model.predict(x_te[ip_features])
        test_ip_preds=pd.DataFrame(np.concatenate((test_ip_preds,x_te[v_features].to_numpy()),axis=1),
                              columns=v_features+["i","p"])
        
        vmp_test_r2,vmp_train_r2,adj_vmp_train_r2,adj_vmp_test_r2,pred_time=evaluate(v_reg_model,ip_preds,y_tr[["Vmp (V)"]],
                                                                           test_ip_preds,y_te[["Vmp (V)"]])

        vmp_test_r2_ls.append(vmp_test_r2)
        vmp_train_r2_ls.append(vmp_train_r2)
        adj_vmp_train_r2_ls.append(adj_vmp_train_r2)
        adj_vmp_test_r2_ls.append(adj_vmp_test_r2)
        pred_time_ls.append(pred_time)
        if(stat_rep):
            v_reg_model.named_steps["model"].get_f_test()
        print("-"*100)
        
        if(solar_model == "mSi460BB" and stat_rep):
            names=get_f_names(v_reg_model)
            print(v_reg_model.named_steps["model"].get_summary(names))
        if(save==True):
            dump(reg_model, f'./Best Models/{solar_model}_{name}_i_reg_model.joblib')
            dump(v_reg_model, f'./Best Models/{solar_model}_{name}_v_model.joblib')
    print(f"mean test r2 : {np.mean(vmp_test_r2_ls)} | mean train r2 :{np.mean(vmp_train_r2_ls)}")  
    print(f"mean test adj-r2 : {np.mean(adj_vmp_test_r2_ls)} | mean train adj-r2 :{np.mean(adj_vmp_train_r2_ls)}") 
    print(f"avg prediction time : {np.mean(pred_time_ls)} seconds")
#aSiTandem90-31 # aSiMicro03038

In [17]:
class StatsModelsOLS(BaseEstimator, RegressorMixin):
    def __init__(self):
        pass
    def fit(self, X, y):
        X_with_const = sm.add_constant(X, has_constant='add')
        self.results_= sm.OLS(y, X_with_const).fit()
        params =self.results_.params
        # print(params.shape)
        self.intercept_=params[0]
        self.coef_ = np.asarray(params[1:])
        return self

    def predict(self, X):
        return self.intercept_ + np.dot(X, self.coef_)
    def get_f_test(self):
        f_statistic = self.results_.fvalue
        f_pvalue= self.results_.f_pvalue
        print(f"f statistics : {f_statistic} | f_pvalue : {f_pvalue}")
    def get_summary(self,names):
        return self.results_.summary(xname=["int"]+names.tolist())
def get_f_names(pipe):
    feat_names_pre=pipe.named_steps["preproc"].get_feature_names_out()
    if("poly" in pipe.named_steps):
        feat_names_pre = pipe.named_steps["poly"].get_feature_names_out(feat_names_pre)
    return feat_names_pre

In [18]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", LinearRegression())
])

In [59]:
# function_set = ['add', 'sub', 'mul', 'div','cos','sin','neg','inv']
# v_symb_reg_model=Pipeline([
#     ("preproc", v_preprocessor),
#     ("model", SymbolicRegressor(
#         function_set=function_set,
#         verbose=1,
#         population_size=100, 
#         generations=20,
#         stopping_criteria=0.01,
#         p_crossover=0.7,
#         p_subtree_mutation=0.1,
#         p_hoist_mutation=0.05, 
#         p_point_mutation=0.1,
#         max_samples=0.9,
#     ))
# ])
# multi_train(reg_model,v_symb_reg_model,stat_rep=False)

In [22]:
# v_ada_model=Pipeline([
#     ("preproc", v_preprocessor),
#     ("model", AdaBoostRegressor(
#         estimator=DecisionTreeRegressor(max_depth=4),
#         n_estimators=100,
#         learning_rate=0.5
#     ))
# ])
# multi_train(reg_model,v_ada_model,stat_rep=False)

In [19]:
v_hist_model=Pipeline([
    ("preproc", v_preprocessor),
    ("model", HistGradientBoostingRegressor(
        max_iter=200,
        learning_rate=0.1,
        max_depth=None,
        random_state=42
    ))
])
multi_train(reg_model,v_hist_model,name="hist",stat_rep=False,save=True)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9926567805630209 | Train R2 : 0.998452051769713
Test ADJ-R2 : 0.99260329753726 | Train ADJ-R2 : 0.998449251067288
predcition time : 0.027159143006429076 seconds
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9897875680423927 | Train R2 : 0.9978449375237197
Test ADJ-R2 : 0.9897139383959788 | Train ADJ-R2 : 0.9978410767883628
predcition time : 0.028725325013510883 seconds
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.7475488067368949 | Train R2 : 0.9814481317195908
Test ADJ-R2 : 0.7457430042099772 | Train ADJ-R2 : 0.98141515648269
predcition time : 0.029597565007861704 seconds
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.8263979272

In [22]:
v_forest_model=Pipeline([
    ("preproc", v_preprocessor),
    ("model",  RandomForestRegressor(
        n_estimators=100,
        max_depth=7,
        min_samples_leaf=1,
        max_features="sqrt",
        bootstrap=True,
        random_state=42)
    )
])
multi_train(reg_model,v_forest_model,stat_rep=False)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9611387791382374 | Train R2 : 0.9733065101733341
Test ADJ-R2 : 0.9608557403846921 | Train ADJ-R2 : 0.9732582136475034
predcition time : 0.02291296102339402 seconds
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9609024679399962 | Train R2 : 0.9670756801448545
Test ADJ-R2 : 0.9606205823447547 | Train ADJ-R2 : 0.9670166971282741
predcition time : 0.022001422999892384 seconds
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.5957610935221135 | Train R2 : 0.7467255384820419
Test ADJ-R2 : 0.5928695419736307 | Train ADJ-R2 : 0.7462753528056858
predcition time : 0.026113198953680694 seconds
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.76465

In [24]:
v_reg_model=Pipeline([
    ("preproc", v_preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", StatsModelsOLS()) #XGBRegressor(**l2_xg_boost_params)
])
multi_train(reg_model,v_reg_model,stat_rep=False)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9569205497675896 | Train R2 : 0.9529339171595328
Test ADJ-R2 : 0.9566067882946659 | Train ADJ-R2 : 0.9528487605052167
predcition time : 0.00843170698499307 seconds
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.935207471779459 | Train R2 : 0.9362047191371987
Test ADJ-R2 : 0.9347403302638098 | Train ADJ-R2 : 0.9360904316401317
predcition time : 0.006979483994655311 seconds
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.4589788147924604 | Train R2 : 0.5151282920633443
Test ADJ-R2 : 0.45510884923303596 | Train ADJ-R2 : 0.5142664511320669
predcition time : 0.005716283980291337 seconds
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.76826

In [25]:
v_reg_model=Pipeline([
    ("preproc", v_preprocessor),
    # ("poly",PolynomialFeatures(degree=2)),
    ("model", XGBRegressor(**l1_xg_boost_params)) #XGBRegressor(**l2_xg_boost_params)
])
multi_train(reg_model,v_reg_model)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9710387146832818 | Train R2 : 0.9755801652607007
Test ADJ-R2 : 0.9708277803401156 | Train ADJ-R2 : 0.975535982458567
predcition time : 0.009123619005549699 seconds
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9718295513968819 | Train R2 : 0.9730205113177959
Test ADJ-R2 : 0.9716264479462465 | Train ADJ-R2 : 0.9729721783033168
predcition time : 0.009400719020050019 seconds
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.5221660548884302 | Train R2 : 0.6533217012123513
Test ADJ-R2 : 0.5187480724484332 | Train ADJ-R2 : 0.652705493784716
predcition time : 0.008752159017603844 seconds
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.763063

In [17]:
v_features=["GHI","DNI","POA_CMP22","Cab_Temp","Dry_Temp","Mod_Temp","Pressure","RH","QA_residual"]

preprocessor = ColumnTransformer([
    ("norm", StandardScaler(),v_features)
])
def single_train(reg_model,stat_rep=False):
    vmp_test_r2_ls=[]
    vmp_train_r2_ls=[]
    adj_vmp_test_r2_ls=[]
    adj_vmp_train_r2_ls=[]
    for solar_model in x_train_sites:
        x_tr=x_train_sites[solar_model]
        x_te=x_test_sites[solar_model]
        y_tr=y_train_sites[solar_model]
        y_te=y_test_sites[solar_model]

        reg_model.fit(x_tr[v_features], y_tr["Vmp (V)"])
    
        print(f"for site {solar_model}")
        
        vmp_test_r2,vmp_train_r2,adj_vmp_train_r2,adj_vmp_test_r2=evaluate(reg_model,x_tr[v_features],y_tr[["Vmp (V)"]],x_te[v_features],y_te[["Vmp (V)"]])
        vmp_test_r2_ls.append(vmp_test_r2)
        vmp_train_r2_ls.append(vmp_train_r2)
        adj_vmp_train_r2_ls.append(adj_vmp_train_r2)
        adj_vmp_test_r2_ls.append(adj_vmp_test_r2)
        if(stat_rep):
            reg_model.named_steps["model"].get_f_test()

        if(solar_model == "aSiTandem90-31" and stat_rep):
            names=get_f_names(reg_model)
            print(reg_model.named_steps["model"].get_summary(names))
            
        print("-"*100)

    print(f"mean test r2 : {np.mean(vmp_test_r2_ls)} | mean train r2 :{np.mean(vmp_train_r2_ls)}")
    print(f"mean test adj-r2 : {np.mean(adj_vmp_test_r2_ls)} | mean train adj-r2 :{np.mean(adj_vmp_train_r2_ls)}") 

In [31]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    ("poly",PolynomialFeatures(degree=2)),
    ("model", StatsModelsOLS()) #XGBRegressor(**l2_xg_boost_params)
])
single_train(reg_model,stat_rep=True)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9335155979665721 | Train R2 : 0.9230628440229718
Test ADJ-R2 : 0.9330801106170081 | Train ADJ-R2 : 0.9229375845432697
f statistics : 1218.2028954781945 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.8957030864912889 | Train R2 : 0.8971999363898134
Test ADJ-R2 : 0.8950268096745898 | Train ADJ-R2 : 0.8970342189310114
f statistics : 895.06596289449 | f_pvalue : 0.0
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.45007194083651403 | Train R2 : 0.4979513237129337
Test ADJ-R2 : 0.4465341620427532 | Train ADJ-R2 : 0.49714833133927394
f statistics : 102.52673732614367 | f_pvalue : 0.0
                            OLS Regression Results                            
Dep. Variable:                Vmp (V)   R-squared:             

In [18]:
reg_model=Pipeline([
    ("preproc", preprocessor),
    # ("poly",PolynomialFeatures(degree=2)),
    ("model", XGBRegressor(**l1_xg_boost_params)) #XGBRegressor(**l2_xg_boost_params)
])
single_train(reg_model)

for site mSi0247
R2 For Vmp (V) : 
Test R2 : 0.9687501756771627 | Train R2 : 0.9740126215795535
Test ADJ-R2 : 0.9685454825047424 | Train ADJ-R2 : 0.9739703121718501
----------------------------------------------------------------------------------------------------
for site CIGS1-001
R2 For Vmp (V) : 
Test R2 : 0.9668904215781657 | Train R2 : 0.9691280419455031
Test ADJ-R2 : 0.9666757341100126 | Train ADJ-R2 : 0.9690782752210735
----------------------------------------------------------------------------------------------------
for site aSiTandem90-31
R2 For Vmp (V) : 
Test R2 : 0.5248470264483382 | Train R2 : 0.6365279173519598
Test ADJ-R2 : 0.5217902882339245 | Train ADJ-R2 : 0.6359465687214583
----------------------------------------------------------------------------------------------------
for site xSi11246
R2 For Vmp (V) : 
Test R2 : 0.7606416204937476 | Train R2 : 0.8777849561481967
Test ADJ-R2 : 0.7590749111006158 | Train ADJ-R2 : 0.8775860528218556
---------------------------